<a href="https://colab.research.google.com/github/muthiearml/MachineLearning/blob/main/Week7/(Lecture7)XGBOOST_Housing_Market_%26_Prices_MuthieArmaliaS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###NAMA : Muthie Armalia S
###NIM  : 1103200178

In [1]:
!pip install kaggle

In [2]:
from google.colab import files

# Upload file kaggle.json
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muthiearmalias","key":"c8aa4051770e2bca69229eaaa6bcdf0c"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets list -s "housing market & prices"

ref                                                     title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rukenmissonnier/final-house                              Housing Market & Prices                            144KB  2023-09-22 08:11:29           1623         36  1.0              
andradaolteanu/redfin-housing-market-prices             [REDFIN] US Housing Market Prices                   856MB  2022-07-25 07:59:19            321         15  1.0              
zillow/zecon                                            Zillow Economics Data                               511MB  2018-01-24 23:53:58          20686        615  0.7058824        
juhibhojani/house-price                                 House Price                                 

In [5]:
# Download dan ekstrak dataset, dimana nantinya akan satu direktori dengan notebook ini
!kaggle datasets download rukenmissonnier/final-house --unzip

  0% 0.00/144k [00:00<?, ?B/s]
100% 144k/144k [00:00<00:00, 69.4MB/s]


In [6]:
# import library

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [7]:
# load data

data = pd.read_csv("house.csv")
data

,bedroom_count,net_sqm,center_distance,metro_distance,floor,age,price
0,1,26.184098,1286.68,204.003817,22,67,96004.804557
1,1,34.866901,1855.25,186.980360,8,30,92473.722568
2,1,36.980709,692.09,111.224999,24,24,98112.519942
3,1,17.445723,1399.49,237.998760,1,66,92118.326874
4,1,52.587646,84.65,100.996400,20,3,98976.653176
...,...,...,...,...,...,...,...
4303,9,175.166533,1299.71,217.739012,2,5,102707.356224
4304,6,215.352151,1346.36,77.724676,14,5,99695.064992
4305,5,299.563972,1585.37,235.537881,3,5,93803.715617
4306,1,29.193907,1896.23,147.223827,4,2,92914.729126


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308 entries, 0 to 4307
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bedroom_count    4308 non-null   int64  
 1   net_sqm          4308 non-null   float64
 2   center_distance  4308 non-null   float64
 3   metro_distance   4308 non-null   float64
 4   floor            4308 non-null   int64  
 5   age              4308 non-null   int64  
 6   price            4308 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 235.7 KB


In [9]:
# Periksa jumlah data yang hilang (NaN) dalam setiap kolom
missing_data = data.isnull().sum()

# Tampilkan data yang hilang
print(missing_data)

bedroom_count      0
net_sqm            0
center_distance    0
metro_distance     0
floor              0
age                0
price              0
dtype: int64


In [10]:
data.shape

(4308, 7)

In [11]:
data = data.copy()  # Membuat salinan DataFrame
data['center_distance'] = data['center_distance'].astype(int)
data['metro_distance'] = data['metro_distance'].astype(int)

In [12]:
# Select subset of predictors
cols_to_use = ['bedroom_count','net_sqm', 'center_distance' ,'metro_distance', 'floor', 'age']
X = data[cols_to_use]

In [13]:
# Select target
y = data.price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42,test_size=0.3)

In [14]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
predictions = my_model.predict(X_valid)
print("Mean Absolute Errors : " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Errors : 1712.046859847072


In [16]:
from sklearn.model_selection import GridSearchCV
parameter = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.1, 0.3, 0.5],
    'max_depth': [3, 5, 7]
}

# Inisialisasi model XGBRegressor
model_0 = XGBRegressor()

# Grid search
grid_search = GridSearchCV(estimator=model_0, param_grid=parameter, cv=5)
grid_search.fit(X_train, y_train)

# Model terbaik
best_model = grid_search.best_estimator_

# Prediksi pada data validasi
predictions = best_model.predict(X_valid)

# Mean absolute error
mae = mean_absolute_error(predictions, y_valid)

# Print mean absolute error
print('Mean absolute error:', mae)

Mean absolute error: 1548.3427843380537
